### my first kaggle! woohoo!
#### first iteration of the kaggle titanic training problem. <3 here goes!

In [101]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import cross_validation
from sklearn.cross_validation import KFold

In [102]:
titanic = pd.read_csv('titanic_train.csv')
titanic.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,30.0708,NaN,C


In [103]:
# get characteristic of the numeric columns
titanic.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


#### looking at the columns, i see there is a different value for 'Age' vs passenger id. these should be the same, but apparently we don't have ages for every passenger. because i need all the data i can get to train, i'm going to clean this up so my rows are still usable.

In [104]:
# replace missing values in 'Age' column with the median for the column, assign it back to the column to replace

titanic['Age'] = titanic['Age'].fillna(titanic['Age'].median())
titanic.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.361582,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,13.019697,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


#### cool, now we've got all our rows.
## getting rid of non-numeric info
#### machine learning algorithms don't do well with non-numeric values. so in order to process them i'll need to convert them. columns like 'Name' would be difficult to convert to a numerical code, and probably don't have a lot of useful information in them anyway (without further information). however gender might, and it's a binary choice which means it'll be easy to convert to something a machine can understand.

In [105]:
# change values in 'Sex' column to 0 (m) and 1 (f)

titanic.loc[titanic['Sex'] == 'male', 'Sex'] = 0
titanic.loc[titanic['Sex'] == 'female', 'Sex'] = 1
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",1,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",0,35,0,0,373450,8.0500,NaN,S


## going to work on the 'Embarked' column now
#### first get all the unique values, then replace missing ones with the most commonly occurring value

In [106]:
titanic['Embarked'].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [107]:
# replace NaN values with the most commonly occuring value--since it's non-numerical, mode instead of mean

#titanic['Embarked'] = titanic['Embarked'].fillna(titanic['Embarked'].mode())
titanic["Embarked"] = titanic["Embarked"].fillna("S")
titanic.head(15)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",1,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",0,35,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",0,28,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",0,54,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",0,2,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",1,27,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",1,14,1,0,237736,30.0708,NaN,C


In [108]:
# next convert to numerical values

embarked_codes = {'S': 0, 'C':1, 'Q':2}

titanic['Embarked'] = titanic['Embarked'].map(embarked_codes)

titanic.head(40)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22,1,0,A/5 21171,7.2500,NaN,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38,1,0,PC 17599,71.2833,C85,1
2,3,1,3,"Heikkinen, Miss. Laina",1,26,0,0,STON/O2. 3101282,7.9250,NaN,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35,1,0,113803,53.1000,C123,0
4,5,0,3,"Allen, Mr. William Henry",0,35,0,0,373450,8.0500,NaN,0
5,6,0,3,"Moran, Mr. James",0,28,0,0,330877,8.4583,NaN,2
6,7,0,1,"McCarthy, Mr. Timothy J",0,54,0,0,17463,51.8625,E46,0
7,8,0,3,"Palsson, Master. Gosta Leonard",0,2,3,1,349909,21.0750,NaN,0
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",1,27,0,2,347742,11.1333,NaN,0
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",1,14,1,0,237736,30.0708,NaN,1


#### yay done! looks legit to me
#### data is now ready to apply machine learning algorithms (:

## linear regression 
#### this is a small dataset, so i'll use KFold to cross-validate and avoid overfit

In [109]:
titanic.shape

(891, 12)

In [110]:
# these are the columns the algorithm will learn from 
predict_cols = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Sex', 'Embarked', 'Age']

# initialize regressor class
regressor = LinearRegression()

# get mah folds
# these correspond to train, test
# taking 3, 2:3 train to test ratio
kfolds = KFold(titanic.shape[0], n_folds=3, random_state=1)

# initialize a list to store predictions
predictions = []

for train, test in kfolds:
    # only using the test folds
    # KFold returns the row indeces for folds, so i'll use iloc[]
    train_folds = titanic[predict_cols].iloc[train,:]
    # training target from the test folds
    train_target = titanic['Survived'].iloc[train]
    
    # train the algorithm
    regressor.fit(train_folds, train_target)
    # make predictions
    # got to tell it to test specifically on rows from the test section
    test_predictions = regressor.predict(titanic[predict_cols].iloc[test,:])
    predictions.append(test_predictions)

# so this goes a total of 3 times
# each cross-validation produces a list of predictions on that iteration's test fold
# these 3 lists are stored in 'predictions', a numpy array
predictions = np.concatenate(predictions, axis=0)

# convert predictions into 1 or 0 
predictions[predictions >= .5] = 1
predictions[predictions <= .5] = 0

# now to test the error
# i'm doing this by kaggle rules, so it's the percentage of correct predictions
# this is going to give me a FutureWarning which will annoy me so i'll suppress that first
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# now calc error
accuracy = sum(predictions[predictions == titanic['Survived']]) / len(predictions)

print(accuracy)

0.783389450056


#### 78% percent is not super great accuracy. time for another method.
## logistic regression and the logit function


In [111]:
from sklearn.linear_model import LogisticRegression

# already imported cross_validation from sklearn
# initialize algorithm class, parameter is for cross validation folds
logit_regress = LogisticRegression(random_state=1)
# get accuracy/error for all (3) folds
# sklearn has helpful functions for this
results = cross_validation.cross_val_score(logit_regress, titanic[predict_cols], titanic['Survived'], cv=3)
# 3 sets of results, so taking the average
print(results.mean())

0.787878787879


#### similar results. ~.78 is not great accuracy. but it's probably the best i'm going to get using these methods.
## final preparations, submission
#### if i were going to submit this to kaggle for their open titanic competition, i'd need to run my algorithm on the test data, then  generate a dataframe with just the pertinenet columns, and output that to a csv file.
#### linear vs logistic regression aren't very different in terms of accuracy here...but i'm going to use my logistic regression for this, because reasons

In [112]:
# from the beginning:

# first need to clean the test data so it's standardized with the training data
# i'll use the exact same processes to make sure everything is the same

titanic_test = pd.read_csv('test.csv')

# change the 'Sex' values to 1 and 0, 'Embarked' codes to numerical (first fill NaNs), and replace 
# 'Fare' and 'Age' NaNs with median values

titanic_test['Age'] = titanic_test['Age'].fillna(titanic['Age'].median())

titanic_test.loc[titanic_test['Sex'] == 'male', 'Sex'] = 0
titanic_test.loc[titanic_test['Sex'] == 'female', 'Sex'] = 1

titanic_test['Embarked'] = titanic_test['Embarked'].fillna('S')
titanic_test.loc[titanic_test['Embarked'] == 'S', 'Embarked'] = 0
titanic_test.loc[titanic_test['Embarked'] == 'C', 'Embarked'] = 1
titanic_test.loc[titanic_test['Embarked'] == 'Q', 'Embarked'] = 2

# using the median from the test DF column, not the origianl, because these may be slightly different
# so the numbers will be different, but the method is consistent. 
titanic_test['Fare'] = titanic_test['Fare'].fillna(titanic_test['Fare'].median())


# initialize algorithm class
logit_regress = LogisticRegression(random_state=1)

logit_regress.fit(titanic[predict_cols], titanic['Survived'])
predictions = logit_regress.predict(titanic_test[predict_cols])

# kaggle wants passenger id and survival predictions in a DF
# so i'll make a new one with the IDs and predictions from the test set
final_df = pd.DataFrame({
        'PassengerId': titanic_test['PassengerId'], 
        'Survived': predictions
    })


# output a csv, ready to submit
final_df.to_csv('kaggle_titanic.csv', index=False)

## ~the end~ [:
#### that's it! we have a model...although it isn't a super accurate model, it does illustrate some basic methods for cleaning data and setting up algorithms. more on the way....